In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [7]:
KEY=os.getenv("OPEN_API_KEY")

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
print(KEY)

sk-proj-WIZT56mbAbTnjlscJAYMT3BlbkFJaoT4i7TmomfV6e1gROzg


In [24]:
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.5,
    openai_api_key=KEY
)


In [25]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000023647D2AF10>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000023647D468D0>, temperature=0.5, openai_api_key='sk-proj-WIZT56mbAbTnjlscJAYMT3BlbkFJaoT4i7TmomfV6e1gROzg', openai_proxy='')

In [26]:
from langchain.llms import OpenAI

In [27]:
from langchain.prompts import PromptTemplate

In [28]:
from langchain.chains import LLMChain

In [29]:
from langchain.chains import SequentialChain

In [30]:
from langchain.callbacks import get_openai_callback

In [31]:
import PyPDF2

In [32]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [34]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [35]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [36]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [37]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [38]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject","quiz"],template=TEMPLATE)

In [39]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [40]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz","review"],verbose=True,)

In [50]:
file_path=r"C:\Users\Lenovo\Desktop\interviewQuestion\mcqgen\data.txt"

In [51]:
file_path

'C:\\Users\\Lenovo\\Desktop\\interviewQuestion\\mcqgen\\data.txt'

In [52]:
with open(file_path,'r') as file:
    TEXT=file.read()

In [53]:
print(TEXT)

ndian music ranges over various traditions and regional styles. Classical music encompasses two genres and their various folk offshoots: the northern Hindustani and the southern Carnatic schools.[432] Regionalised popular forms include filmi and folk music; the syncretic tradition of the bauls is a well-known form of the latter. Indian dance also features diverse folk and classical forms. Among the better-known folk dances are: bhangra of Punjab, bihu of Assam, Jhumair and chhau of Jharkhand, Odisha and West Bengal, garba and dandiya of Gujarat, ghoomar of Rajasthan, and lavani of Maharashtra. Eight dance forms, many with narrative forms and mythological elements, have been accorded classical dance status by India's National Academy of Music, Dance, and Drama. These are: bharatanatyam of the state of Tamil Nadu, kathak of Uttar Pradesh, kathakali and mohiniyattam of Kerala, kuchipudi of Andhra Pradesh, manipuri of Manipur, odissi of Odisha, and the sattriya of Assam.[433]

Theatre in I

In [54]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [57]:
NUMBER=5 
SUBJECT="India"
TONE="simple"

In [58]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:ndian music ranges over various traditions and regional styles. Classical music encompasses two genres and their various folk offshoots: the northern Hindustani and the southern Carnatic schools.[432] Regionalised popular forms include filmi and folk music; the syncretic tradition of the bauls is a well-known form of the latter. Indian dance also features diverse folk and classical forms. Among the better-known folk dances are: bhangra of Punjab, bihu of Assam, Jhumair and chhau of Jharkhand, Odisha and West Bengal, garba and dandiya of Gujarat, ghoomar of Rajasthan, and lavani of Maharashtra. Eight dance forms, many with narrative forms and mythological elements, have been accorded classical dance status by India's National Academy of Music, Dance, and Drama. These are: bharatanatyam of the state of Tamil Nadu, kathak of Uttar Pradesh, kathakali and mohiniyattam of Kerala, kuchi

In [59]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:5430
Prompt Tokens:4754
Completion Tokens:676
Total Cost:0.008483


In [60]:
response

{'text': 'ndian music ranges over various traditions and regional styles. Classical music encompasses two genres and their various folk offshoots: the northern Hindustani and the southern Carnatic schools.[432] Regionalised popular forms include filmi and folk music; the syncretic tradition of the bauls is a well-known form of the latter. Indian dance also features diverse folk and classical forms. Among the better-known folk dances are: bhangra of Punjab, bihu of Assam, Jhumair and chhau of Jharkhand, Odisha and West Bengal, garba and dandiya of Gujarat, ghoomar of Rajasthan, and lavani of Maharashtra. Eight dance forms, many with narrative forms and mythological elements, have been accorded classical dance status by India\'s National Academy of Music, Dance, and Drama. These are: bharatanatyam of the state of Tamil Nadu, kathak of Uttar Pradesh, kathakali and mohiniyattam of Kerala, kuchipudi of Andhra Pradesh, manipuri of Manipur, odissi of Odisha, and the sattriya of Assam.[433]\n\

In [62]:
quiz=response.get("quiz")

In [64]:
json.loads(quiz)

{'1': {'mcq': 'What is the traditional dress worn by women in India?',
  'options': {'a': 'Jeans and sweater',
   'b': 'Churidars and kameez',
   'c': 'Bandgala and Nehru jackets',
   'd': 'Dhoti and woollen shawl'},
  'correct': 'b'},
 '2': {'mcq': "Which dance form has been accorded classical dance status by India's National Academy of Music, Dance, and Drama?",
  'options': {'a': 'Bhangra',
   'b': 'Garba',
   'c': 'Bharatanatyam',
   'd': 'Ghoomar'},
  'correct': 'c'},
 '3': {'mcq': 'What is the largest sector of the Indian film industry in terms of box office revenue?',
  'options': {'a': 'Bengali film industry',
   'b': 'Telugu film industry',
   'c': 'Hindi film industry (Bollywood)',
   'd': 'Tamil film industry'},
  'correct': 'c'},
 '4': {'mcq': 'Which garment is traditionally worn by men in India?',
  'options': {'a': 'Sari', 'b': 'Kurta', 'c': 'Choli', 'd': 'Pyjamas'},
  'correct': 'b'},
 '5': {'mcq': 'What is the literacy rate of the Indian population according to the 2011